In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import date

In [36]:
df_train=pd.read_csv('train.csv')

In [37]:
df_train=df_train.dropna()

In [38]:
df_train.shape

(225493, 41)

In [39]:
df_train.shape

(225493, 41)

In [40]:
df_train.select_dtypes('object').columns

Index(['Date.of.Birth', 'Employment.Type', 'DisbursalDate',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'AVERAGE.ACCT.AGE',
       'CREDIT.HISTORY.LENGTH'],
      dtype='object')

In [41]:
def cns_desc(x):
    if x<300:
        return 0
    elif (x>=300) and (x<=350):
        return 1
    elif (x>350) and (x<=570):
        return 2
    elif (x>570) and (x<=630):
        return 3
    elif (x>630) and (x<=705):
        return 4
    else:
        return 5

In [42]:
def fix_date(x):

    if x.year >= 2049:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)
df_train["Date.of.Birth"]=pd.to_datetime(df_train["Date.of.Birth"])
df_train["Date.of.Birth"] = df_train["Date.of.Birth"].apply(fix_date)



In [43]:
df_train["Date.of.Birth"]=pd.to_datetime(df_train["Date.of.Birth"])

df_train["DisbursalDate"]=pd.to_datetime(df_train["DisbursalDate"])
df_train["DisbursalDate_Month"]=df_train["DisbursalDate"].dt.month

df_train["Age_at_disbursement"]=((df_train['DisbursalDate']-df_train["Date.of.Birth"])/365).apply(lambda x:x.days)

df_train["Employment.Type"]=df_train["Employment.Type"].map({'Salaried':0,"Self employed":1,np.nan:2})

df_train["PERFORM_CNS.SCORE.DESCRIPTION"]=df_train["PERFORM_CNS.SCORE"].apply(cns_desc)

df_train["AVERAGE.ACCT.AGE"]=df_train["AVERAGE.ACCT.AGE"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

df_train["CREDIT.HISTORY.LENGTH"]=df_train["CREDIT.HISTORY.LENGTH"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

In [44]:
df_train['NO.OF.ACCTS']=df_train['PRI.NO.OF.ACCTS']+df_train['SEC.NO.OF.ACCTS']
df_train['ACTIVE.ACCTS']=df_train['PRI.ACTIVE.ACCTS']+df_train['SEC.ACTIVE.ACCTS']
df_train['OVERDUE.ACCTS']=df_train['PRI.OVERDUE.ACCTS']+df_train['SEC.OVERDUE.ACCTS']
df_train['CURRENT.BALANCE']=df_train['PRI.CURRENT.BALANCE']+df_train['SEC.CURRENT.BALANCE']
df_train['SANCTIONED.AMOUNT']=df_train['PRI.SANCTIONED.AMOUNT']+df_train['SEC.SANCTIONED.AMOUNT']
df_train['DISBURSED.AMOUNT']=df_train['PRI.DISBURSED.AMOUNT']+df_train['SEC.DISBURSED.AMOUNT']
df_train['INSTAL.AMT']=df_train['PRIMARY.INSTAL.AMT']+df_train['SEC.INSTAL.AMT']

df_train=df_train.drop(['PRI.NO.OF.ACCTS','SEC.NO.OF.ACCTS','PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','SEC.OVERDUE.ACCTS','PRI.CURRENT.BALANCE','SEC.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','SEC.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT'],axis=1)

In [45]:
df_train=df_train.drop(columns=['Date.of.Birth',"DisbursalDate"])

In [46]:
df_train[df_train['DISBURSED.AMOUNT']==df_train['DISBURSED.AMOUNT'].max()]

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,...,loan_default,DisbursalDate_Month,Age_at_disbursement,NO.OF.ACCTS,ACTIVE.ACCTS,OVERDUE.ACCTS,CURRENT.BALANCE,SANCTIONED.AMOUNT,DISBURSED.AMOUNT,INSTAL.AMT
71593,598208,47773,85964,59.33,20,16700,86,6434,1,5,...,1,10,50,2,1,0,36041,1000000000,1000000000,0


In [47]:
df_train=df_train.drop([71593])

In [48]:
df_train.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,...,loan_default,DisbursalDate_Month,Age_at_disbursement,NO.OF.ACCTS,ACTIVE.ACCTS,OVERDUE.ACCTS,CURRENT.BALANCE,SANCTIONED.AMOUNT,DISBURSED.AMOUNT,INSTAL.AMT
0,420825,50578,58400,89.55,67,22807,45,1441,0,6,...,0,3,34,0,0,0,0,0,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,1,6,...,1,9,33,1,1,1,27600,50200,50200,1991
2,417566,53278,61360,89.63,67,22807,45,1497,1,6,...,0,1,32,0,0,0,0,0,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,1,6,...,1,10,24,3,0,0,0,0,0,31
4,539055,52378,60300,88.39,67,22807,45,1495,1,6,...,1,9,41,0,0,0,0,0,0,0


In [49]:
def dummies_top(ld_all,cat_cols):
    for col in cat_cols:
        freqs=ld_all[col].value_counts()
        k=freqs.index[freqs>20][:-1]
        for cat in k:
            name=col+'_'+str(cat)
            ld_all[name]=(ld_all[col]==cat).astype(int)
        del ld_all[col]
        print(col)

In [50]:
# df_train=df_train.drop(columns=['UniqueID','supplier_id','Current_pincode_ID','Employee_code_ID',"MobileNo_Avl_Flag"])
cols_to_be_dummied=["branch_id","State_ID","manufacturer_id",'Employment.Type','DisbursalDate_Month','PERFORM_CNS.SCORE.DESCRIPTION']

df_train=pd.get_dummies(data=df_train,columns=cols_to_be_dummied,drop_first=True)
# dummies_top(df_train,cols_to_be_dummied)


In [71]:
df_train['ltv']=np.square(df_train['ltv'])

In [72]:
# from sklearn.preprocessing import OneHotEncoder
# for i in ['supplier_id','Current_pincode_ID','Employee_code_ID']:
#     one_hot=OneHotEncoder(handle_unknown="ignore",sparse=False)
#     dummy_index=df_train[i].value_counts()[:100].index.to_list()
#     one_hot.fit(np.array(dummy_index).reshape(-1, 1))
#     data_train=one_hot.transform(df_train[i].values.reshape(-1, 1))
#     dummy_train=pd.DataFrame(data=data_train,columns=[str(j)+i for j in dummy_index],index=df_train.index)
#     df_train=pd.concat([df_train,dummy_train],axis=1)


In [73]:
list1=['disbursed_amount','asset_cost','PERFORM_CNS.SCORE','NEW.ACCTS.IN.LAST.SIX.MONTHS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH','NO.OF_INQUIRIES','NO.OF.ACCTS','ACTIVE.ACCTS','OVERDUE.ACCTS','CURRENT.BALANCE','SANCTIONED.AMOUNT','DISBURSED.AMOUNT','INSTAL.AMT']

In [74]:
from sklearn.preprocessing import MinMaxScaler
mn=MinMaxScaler()
df_train['CURRENT.BALANCE']=mn.fit_transform(df_train['CURRENT.BALANCE'].values.reshape(-1,1))

In [75]:
list_to_apply_log=df_train[list1].skew()[df_train[list1].skew()>2].index

In [76]:
for i in list_to_apply_log:
    df_train[i]=df_train[i]+1
    df_train[i]=np.log(df_train[i])

In [77]:
df_train

,UniqueID,disbursed_amount,asset_cost,ltv,supplier_id,Current_pincode_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,...,DisbursalDate_Month_8,DisbursalDate_Month_9,DisbursalDate_Month_10,DisbursalDate_Month_11,DisbursalDate_Month_12,PERFORM_CNS.SCORE.DESCRIPTION_1,PERFORM_CNS.SCORE.DESCRIPTION_2,PERFORM_CNS.SCORE.DESCRIPTION_3,PERFORM_CNS.SCORE.DESCRIPTION_4,PERFORM_CNS.SCORE.DESCRIPTION_5
0,420825,10.831292,10.975088,8019.2025,22807,1441,1998,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,537409,10.761004,11.090584,5362.6329,22807,1502,1998,1,1,0,...,0,1,0,0,0,0,0,1,0,0
2,417566,10.883298,11.024530,8033.5369,22807,1497,1998,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,624493,10.959784,11.099136,7828.7104,22807,1501,1998,1,1,0,...,0,0,1,0,0,1,0,0,0,0
4,539055,10.866261,11.007104,7812.7921,22807,1495,1998,1,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233149,626432,11.054281,11.565575,3686.9184,20700,1050,3705,1,0,0,...,0,0,1,0,0,0,0,0,0,1
233150,606141,11.207107,11.518917,5617.5025,23775,990,3705,1,0,0,...,0,0,1,0,0,0,0,0,0,1
233151,613658,10.418853,11.173431,2347.4025,22186,2299,3479,1,1,0,...,0,0,1,0,0,0,0,0,0,0
233152,548084,10.441734,11.202139,2410.8100,22186,2299,3479,1,1,0,...,0,1,0,0,0,0,0,0,0,0


In [78]:
main_cols=['disbursed_amount','asset_cost','PERFORM_CNS.SCORE','NEW.ACCTS.IN.LAST.SIX.MONTHS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH','NO.OF_INQUIRIES','NO.OF.ACCTS','ACTIVE.ACCTS','OVERDUE.ACCTS','CURRENT.BALANCE','SANCTIONED.AMOUNT','DISBURSED.AMOUNT','INSTAL.AMT']

In [79]:
df_train.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'supplier_id',
       'Current_pincode_ID', 'Employee_code_ID', 'MobileNo_Avl_Flag',
       'Aadhar_flag', 'PAN_flag',
       ...
       'DisbursalDate_Month_8', 'DisbursalDate_Month_9',
       'DisbursalDate_Month_10', 'DisbursalDate_Month_11',
       'DisbursalDate_Month_12', 'PERFORM_CNS.SCORE.DESCRIPTION_1',
       'PERFORM_CNS.SCORE.DESCRIPTION_2', 'PERFORM_CNS.SCORE.DESCRIPTION_3',
       'PERFORM_CNS.SCORE.DESCRIPTION_4', 'PERFORM_CNS.SCORE.DESCRIPTION_5'],
      dtype='object', length=157)

In [80]:
from sklearn.preprocessing import MinMaxScaler

In [81]:
X=df_train.drop(['loan_default'],axis=1)
y=df_train['loan_default']

In [82]:
mn=MinMaxScaler()
new_X=mn.fit_transform(X)

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,KFold
lr=LogisticRegression()

In [84]:
from sklearn.decomposition import PCA
pca = PCA()
new_X = pca.fit_transform(new_X)

In [85]:
AB_bias=[]
AB_var=[]
# for n in np.arange(1,10,3):
# for n in [0.0001,0.0006,0.0002]:
for i in [5000]:
  lr=LogisticRegression(penalty='l2',random_state=1,max_iter=i)
  fold=KFold(n_splits=5,shuffle=True, random_state=1)
  scores=cross_val_score(lr,new_X,y,cv=fold,scoring='roc_auc',n_jobs=-1)
  AB_bias.append(np.mean(scores))
  AB_var.append(np.std(scores,ddof=1))

In [86]:
AB_bias

[0.6556998778814044]

In [87]:
AB_var

[0.002721030998259189]

In [88]:
scores

array([0.65267506, 0.6601115 , 0.65519928, 0.65483203, 0.65568151])

In [89]:
df_train['loan_default'].value_counts(normalize=True)

0    0.782848
1    0.217152
Name: loan_default, dtype: float64

In [91]:
from sklearn.model_selection import train_test_split

In [92]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.3, random_state=1)

In [94]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
pred=lr.predict(X_test)

In [110]:
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report
def model_eval(model,X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    pred_prob=model.predict_proba(X_test)[:,-1]
    print('accuracy:',accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print('roc_auc:',roc_auc_score(y_test, pred_prob))   
    

In [111]:
model_eval(lr,X_train, X_test, y_train, y_test)

accuracy: 0.7822404210028382
              precision    recall  f1-score   support

           0       0.78      1.00      0.88     52932
           1       0.48      0.01      0.02     14716

    accuracy                           0.78     67648
   macro avg       0.63      0.50      0.45     67648
weighted avg       0.72      0.78      0.69     67648

roc_auc: 0.6551383426559665
